In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from astropy.table import Table, vstack
from astropy.utils.data import download_file
from astropy.time import Time

from bs4 import BeautifulSoup
from urllib.request import HTTPError


base_url = "http://mkwc.ifa.hawaii.edu/archive/wx/ukirt/"

In [ ]:
tables = []

for i in range(1991, 2019):
    print('Downloading year:', i)
    try: 
        path = open(download_file(base_url + 'ukirt-wx.{0}.dat'.format(i), cache=True)).read()
        # Files before 2007 have a header row
        if i < 2007: 
            table = Table.read(path, format='ascii')
        
        # The 2007 table is in a different format from the others, skip it
        elif i in [2007, 2008]:
            pass
        
        # Files later than 2007 have no header
        else: 
            table = Table.read(path, format='ascii.fixed_width_no_header', delimiter=' ')
    
    # If the file is inaccessible, skip it and move on
    except HTTPError: 
        pass
        
    tables.append(table)

In [ ]:
for table in tables: 
    if 'year' in table.colnames: 
        time_strings = ["{0}-{1}-{2} {3}:{4}:{5}"
                        .format(row['year'], row['month'], row['day'], 
                                row['hour'], row['min'], row['sec'])
                        for row in table]
        times = Time(time_strings, format='iso')
        table['times'] = times
    else: 
        time_strings = ["{0}-{1:02d}-{2} {3:02d}:{4:02d}"
                        .format(row['col1'], row['col2'], row['col3'], 
                                row['col4'], row['col5']).replace(',', '')
                        for row in table]
        times = Time(time_strings, format='iso')
        table['times'] = times

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

for table in tables: 
    if 'year' in table.colnames: 
        ax.plot_date(table['times'].plot_date, table['air_temp(C)'], marker='.', color='k', alpha=0.2)
    else: 
        # Need to figure out how to handle this case when there are no column headers...
        pass
    
ax.set_ylim([-10, 20])
ax.set_xlabel('Time')
ax.set_ylabel('Air Temp [C]')
ax.set_title('UKIRT')